# Best region for oil well development

We work for the OilyGiant mining company. Our task is to find the best place for a new well.


Steps to choose the location:
- Collect the oil well parameters in the selected region: oil quality and volume of reserves;
- Build a model for predicting the volume of reserves in the new wells;
- Pick the oil wells with the highest estimated values;
- Pick the region with the highest total profit for the selected oil wells.


We have data on oil samples from three regions. Parameters of each oil well in the region are already known. We will build a model that will help to pick the region with the highest profit margin. We will analyze potential profit and risks using the Bootstrapping technique.

**Project goal**: To find the best region for a well development, based on profit and risk analysis of three regions, and provide a model to choose top 200 wells from 500 wells samples. 

## Loading and viewing data

In [51]:
# Loading libraries
from scipy import stats as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import warnings
warnings.filterwarnings("ignore")
# pd.options.display.float_format = '{:.3f}'.format

In [52]:
# load the data into DataFrames: 
df0 = pd.read_csv('/datasets/geo_data_0.csv')
df1 = pd.read_csv('/datasets/geo_data_1.csv')
df2 = pd.read_csv('/datasets/geo_data_2.csv')

In [53]:
# Look at region 0
df0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [54]:
# Look at region 1
df1

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [55]:
# Look at region 2
df2

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


In [56]:
# Genaral info of region 0 
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [57]:
# Genaral info of region 1
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [58]:
# Genaral info of region 2
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Each region has full 100K entries, each with 5 columns: well id, 3 point features (marked f0, f1, f2), and the target - the product (volume of oil reserves, in thousand barrels). We will make the following **preparations** (for each region): 

1. Define features(X) and target(y): features will be the three features ('id' has no relevance for predicting oil volume), target will be the product.
2. Split the data into a training set and validation set at a ratio of 75:25.
3. Standardize the 3 features (scale, all 3 are numeric)

These preparations will be included in a unified function, that will also include training a linear regression model, and predicting a well's volume of oil reserves (based on it's 3 features).  

## Train and test the model for each region:

### Split the data into training and validation sets, train the model and make predictions for the validation set - preparation 

In [59]:
# A unified function to split, scale, train on linear regression, and predict (later to be applied on each region=df) 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

def split_scale_train_predict (df):
    # We want to split the data in 75:25 for train:valid dataset
    train_size=0.75
    
    # For convenience we will symbol 'features' as 'X', and 'target' as 'y'
    X = df.drop(['id', 'product'], axis=1)
    y = df['product']

    # We will split the data in training and validation datasets
    X_train, X_valid, y_train, y_valid = train_test_split(X,y, train_size=0.75, random_state=12345)
    
    # Standardize the numeric features:
    # Create an instance of the StandardScaler() class and tune it using the training data.
    scaler = StandardScaler()
    scaler.fit(X_train)
    # Transform train and validation data: 
    X_train = scaler.transform(X_train)
    X_valid = scaler.transform(X_valid)
    
    # Define the model
    model = LinearRegression() 
    # Train the model on the training set:
    model.fit(X_train, y_train)
    # Predict on validation set: 
    predicted_valid = model.predict(X_valid)
    
    # the return values of the function:
    return predicted_valid, y_valid  

### Apply and save the predictions and correct answers for the validation set, and print the average volume of predicted reserves and model RMSE

In [60]:
# Apply the function to region 0, save predictions and correct answers (validation set)
# and calculate average volume of predicted reserves and model RMSE:
from sklearn.metrics import mean_squared_error

predictions_region_0, correct_answers_region_0 = split_scale_train_predict (df0)

# Calculate average volume of predicted reserves:
av_pred_0 = predictions_region_0.mean()
# Calculate RMSE:
rmse_0 = mean_squared_error(correct_answers_region_0, predictions_region_0) ** 0.5  

print('Average volume of predicted reserves Region 0 =', av_pred_0)
print('RMSE of Region 0 =', rmse_0)

Average volume of predicted reserves Region 0 = 92.59256778438035
RMSE of Region 0 = 37.5794217150813


In [61]:
# Apply the function to region 1, save predictions and correct answers (validation set)
# and calculate average volume of predicted reserves and model RMSE:
from sklearn.metrics import mean_squared_error

predictions_region_1, correct_answers_region_1 = split_scale_train_predict (df1)

# Calculate average volume of predicted reserves:
av_pred_1 = predictions_region_1.mean()
# Calculate RMSE:
rmse_1 = mean_squared_error(correct_answers_region_1, predictions_region_1) ** 0.5  

print('Average volume of predicted reserves Region 1 =', av_pred_1)
print('RMSE of Region 1 =', rmse_1)

Average volume of predicted reserves Region 1 = 68.728546895446
RMSE of Region 1 = 0.893099286775617


In [62]:
# Apply the function to region 2, save predictions and correct answers (validation set)
# and calculate average volume of predicted reserves and model RMSE:
from sklearn.metrics import mean_squared_error

predictions_region_2, correct_answers_region_2 = split_scale_train_predict (df2)

# Calculate average volume of predicted reserves:
av_pred_2 = predictions_region_2.mean()
# Calculate RMSE:
rmse_2 = mean_squared_error(correct_answers_region_2, predictions_region_2) ** 0.5  

print('Average volume of predicted reserves Region 2 =', av_pred_2)
print('RMSE of Region 2 =', rmse_2)

Average volume of predicted reserves Region 2 = 94.96504596800489
RMSE of Region 2 = 40.02970873393434


### Analyze the results

When checked on the validation sets (each with 25000 wells) - out of the three regions, two have higher average volume of predicted reserves - region 0 (92.59 product units), and region 2 (94.97 product units), but these regions' RMSEs are pretty high too (37.58 for region 0, 40.03 for region 2). The third region, region 1, has much lower average volume of predicted reserves, only 68.73 product units, but also a very small RMSE (only 0.89). We will compare these results to our results in the following stages, in order to understand their meaning for region choice.     

## Prepare for profit calculation:

We will store all key values for calculations in separate variables. Then we will calculate the volume of reserves sufficient for developing a new well without losses (and the same for all 200 wells). Then we wil compare the obtained value with the average volume of reserves in each region, as we found above.

In [63]:
# Store all key values for calculations in separate variables
wells_budget = 100000000 
wells_count = 200
revenue_of_unit_product = 4500
new_well_cost = wells_budget / wells_count 

# Calculate the volume of reserves sufficient for developing a new well without losses
break_even_well_volume = new_well_cost / revenue_of_unit_product 
print('The volume of reserves sufficient for developing a new well without losses:', break_even_well_volume, 'product units')
print('The volume of reserves sufficient for developing 200 new wells without losses:',
     break_even_well_volume * wells_count, 'product units')

The volume of reserves sufficient for developing a new well without losses: 111.11111111111111 product units
The volume of reserves sufficient for developing 200 new wells without losses: 22222.222222222223 product units


As we can see - the volume of reserves sufficient for developing a new well without losses is more than 111 product units, which is more than 15% higher than the average volume of predicted reserves in regions 0 and 2 (~92-95), and more than 60% (!) higher than the average volume of predicted reserves in region 1 (about 69) we will now try to see what happens when we don't look at total average but only at the selected wells with highest values of predictions.  

## Write a function to calculate profit from a set of selected oil wells and model predictions:

We will pick the wells with the highest values of predictions, then summarize the target volume of reserves in accordance with these predictions. Then we will provide the findings: suggest a region for oil wells' development and justify the choice. And finally calculate the profit for the obtained volume of reserves.

In [64]:
# A function to calculate volume from a set of selected oil wells and model predictions
def volume (predictions, answers, count):
    predictions = pd.Series(predictions, index=answers.index)
    pred_sorted = predictions.sort_values(ascending=False)
    selected = answers[pred_sorted.index][:count]
    return selected.sum()

sum_target_volume_0 = volume (predictions_region_0, correct_answers_region_0, wells_count)

In [65]:
# Apply the function to each region:
sum_target_volume_0 = volume (predictions_region_0, correct_answers_region_0, wells_count)
sum_target_volume_1 = volume (predictions_region_1, correct_answers_region_1, wells_count)
sum_target_volume_2 = volume (predictions_region_2, correct_answers_region_2, wells_count)
print("The sum target volume of 200 highest-predicted wells in region 0 =", sum_target_volume_0, 'product units')
print("The sum target volume of 200 highest-predicted wells in region 1 =", sum_target_volume_1, 'product units')
print("The sum target volume of 200 highest-predicted wells in region 2 =", sum_target_volume_2, 'product units')

The sum target volume of 200 highest-predicted wells in region 0 = 29601.83565142189 product units
The sum target volume of 200 highest-predicted wells in region 1 = 27589.081548181137 product units
The sum target volume of 200 highest-predicted wells in region 2 = 28245.22214133296 product units


As we can see - region 0 has the highest sum target volume of the top 200 highest-predicted volume wells (almost 29602 product units), although all three regions provide volumes way above the 'break-even' threshold of 22222.22 product units (that was calculated above). Now we will calculate the profit for the obtained volume of reserves, for the case of region 0. 

In [66]:
# Calculate the profit for the obtained volume of reserves in region 0:
profit_region_0 = (sum_target_volume_0 * revenue_of_unit_product) - wells_budget 
profit_region_0

33208260.43139851

The profit for the obtained volume of reserves (region 0, 200 top-predicted wells) is 33,208,260 USD. We will move now to risks calculation, also to see if it might change our region's preference.  

## Calculate risks and profit for each region:
Since in reality only 500 wells will be chosen randomly in the chosen region (and the 200 top-predicted wells will be chosen only from these random 500 wells), we will use the bootstrapping technique with 1000 samples to find the distribution of profit in each region, as a critical factor for region choice. 
We will find average profit, 95% confidence interval and risk of losses. Loss is negative profit, we will calculate it as a probability and then express it as a percentage. Management defined that risk of losses must be lower than 2.5%.

### Calculations

First we will define functions to calculate profit and bootstrap for distribution of profit: average, 95% confidence level and risk of losses, and then we will apply it to each region and check and compare results. 

In [67]:
# Functions to calculate profit and bootstrap for distribution of profit: average, 95% confidence level and risk of losses: 
def profit (predictions, answers, count, revenue, budget): 
    profit = (revenue * volume(predictions, answers, count)) - budget 
    return profit 


def bootstrap (predictions, answers):
    state = np.random.RandomState(12345)
    
    profits = []
    for i in range(1000):
        predictions = pd.Series(predictions, index=answers.index)
        target_subsample = answers.sample(n=500, replace=True, random_state=state)
        preds_subsample = predictions[target_subsample.index]
        
        profits.append(profit(preds_subsample, target_subsample, wells_count, revenue_of_unit_product, wells_budget)) 

    profits = pd.Series(profits)
    mean = profits.mean()
    lower = profits.quantile(0.025)
    upper = profits.quantile(0.975) 
    loss = len(profits[profits.values < 0]) / len(profits)
    
    return mean, lower, upper, loss 

In [68]:
# Apply to region 0:
region_0_average_profit, region_0_lower, region_0_upper, region_0_loss = bootstrap(
    predictions_region_0, correct_answers_region_0)
print('Region 0:')
print('Average profit:', region_0_average_profit)
print('95% confidence interval: between', region_0_lower, 'and', region_0_upper)
print('Risk of losses:', region_0_loss * 100, '%')

Region 0:
Average profit: 4259385.269105923
95% confidence interval: between -1020900.9483793724 and 9479763.533583675
Risk of losses: 6.0 %


In [69]:
# Apply to region 1:
region_1_average_profit, region_1_lower, region_1_upper, region_1_loss = bootstrap(
    predictions_region_1, correct_answers_region_1)
print('Region 1:')
print('Average profit:', region_1_average_profit)
print('95% confidence interval: between', region_1_lower, 'and', region_1_upper)
print('Risk of losses:', region_1_loss * 100, '%')

Region 1:
Average profit: 5152227.734432898
95% confidence interval: between 688732.2537050088 and 9315475.912570495
Risk of losses: 1.0 %


In [70]:
# Apply to region 2:
region_2_average_profit, region_2_lower, region_2_upper, region_2_loss = bootstrap(
    predictions_region_2, correct_answers_region_2)
print('Region 2:')
print('Average profit:', region_2_average_profit)
print('95% confidence interval: between', region_2_lower, 'and', region_2_upper)
print('Risk of losses:', region_2_loss * 100, '%')

Region 2:
Average profit: 4350083.627827557
95% confidence interval: between -1288805.473297878 and 9697069.541802654
Risk of losses: 6.4 %


### Conclusion

Analysis shows that both region 0 and region 2 have 6% (or more) risk of losses, and this fact disqualify these two regions from being chosen for well development. **Region 1** has only 1% risk of losses, which meets management demand for risk lower than 2.5%, so **it is our suggestion for well development**. This low risk for losses fits the low RMSE that was found for the prediction model above, to be used with the samples of the random 500 wells in this region, in order to choose best 200 wells out of the 500 based on their 3 features.

The average profit from 200 wells (out of random 500 wells) in region 1 is 5,152,227.73 USD, and we have confidence of 95% that profit will be in the range between 688,732 USD and 9,315,476 USD.   